In [1]:
#Install the packages we need, including Google OR-Tools 
%%capture
!pip install ortools
!pip install pandas
!pip install numpy
!pip install matplotlib

!apt install gdal-bin python-gdal python3-gdal 
%pip install pandas fiona shapely pyproj
!apt install python3-rtree 
!pip install geopandas==0.10.0
!pip install descartes
!pip install scipy 
!pip install statsmodels
!pip install gdal
!pip install scikit-learn
!pip install statsmodels
!pip install networkx
!pip install osmnx

In [2]:
#Here we are importing the packages we need. 
import geopandas as gpd
import pandas as pd 
from geopandas.tools import sjoin
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import shape
from descartes import PolygonPatch
import time
import math
import scipy.stats as stats
import numpy as np
import os, sys
from pyproj import CRS, Transformer
import fiona

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import matplotlib as mpl
from math import floor

from shapely.ops import unary_union

import warnings
warnings.filterwarnings('ignore')

from osgeo import ogr, gdal,osr

from ortools.linear_solver import pywraplp
import string

import networkx as nx
import osmnx as ox

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

#Navigate to folder where data is stored in the drive. 
%cd /content/drive/MyDrive/jameslab/

dirname = '/content/drive/MyDrive/jameslab/'

Mounted at /content/drive
/content/drive/MyDrive/jameslab


In [4]:
def get_potential_sites(gdf,spacing): 

    bounds = gdf.bounds
    xmax = bounds['maxx']
    xmin= bounds['minx']
    ymax = bounds['maxy']
    ymin = bounds['miny']

    num_col = int((xmax - xmin) / spacing) 
    num_row = int((ymax - ymin) / spacing)

    Yi = np.linspace(ymin,ymax,num_row+1)
    Xi = np.linspace(xmin,xmax,num_col+1)
    Xi,Yi = np.meshgrid(Xi,Yi)

    points = [Point(x) for x in zip(Xi.flatten(),Yi.flatten())]
    points = gpd.GeoDataFrame(geometry=gpd.GeoSeries(points),crs='esri:102001')\
    .to_crs('EPSG:4326')

    points['lat'] = list(points['geometry'].y)
    points['lon'] = list(points['geometry'].x)


    return points

In [28]:
# Create a regular grid of points in the area of interest, which is the bounding
#box of the 2020 (maximum) defoliation extent 

sa = gpd.read_file('study_area/area_of_interest.shp').to_crs('esri:102001')
sites_df = get_potential_sites(sa,500)

In [59]:
def join_shapefile(gdf,aerial_survey,col_name,fire=False,harvest=False,road=False): 
    if fire == False and harvest == False and road == False: 
        points_in_shp = sjoin(gdf, aerial_survey, how='left')
        points_in_shp['FOREST_DAM'] = points_in_shp['FOREST_DAM'].fillna(0)
        points_in_shp.loc[points_in_shp['FOREST_DAM'] != 0, 'FOREST_DAM'] = 1
        points_in_shp[col_name] = points_in_shp['FOREST_DAM']

    elif harvest == True and fire == False and road == False: 
        points_in_shp = sjoin(gdf, aerial_survey, how='left')
        points_in_shp['HarvestCon'] = points_in_shp['HarvestCon'].fillna(0)
        points_in_shp.loc[points_in_shp['HarvestCon'] != 0, 'HarvestCon'] = 1
        points_in_shp[col_name] = points_in_shp['HarvestCon']     

    elif road == True and fire == False and harvest == False: 
        points_in_shp = sjoin(gdf, aerial_survey, how='left')
        points_in_shp['featurecla'] = points_in_shp['featurecla'].fillna(0)
        points_in_shp.loc[points_in_shp['featurecla'] != 0, 'featurecla'] = 1
        points_in_shp[col_name] = points_in_shp['featurecla'] 

    else: 
        points_in_shp = sjoin(gdf, aerial_survey, how='left')
        points_in_shp['FIRE_ID'] = points_in_shp['FIRE_ID'].fillna(0)
        points_in_shp.loc[points_in_shp['FIRE_ID'] != 0, 'FIRE_ID'] = 1
        points_in_shp[col_name] = points_in_shp['FIRE_ID']      

    array = np.array(points_in_shp[col_name])
    Xi = np.array(points_in_shp['lon'])
    Yi = np.array(points_in_shp['lat'])

    new_lon = []
    new_lat = []
    new_val = [] 
    for val,lon,lat in zip(array.flatten(),Xi.flatten(),Yi.flatten()):
        new_lon.append(lon)
        new_lat.append(lat)
        new_val.append(val)    

    points = [Point(x) for x in zip(Xi.flatten(),Yi.flatten())]

    #new_data = gpd.GeoDataFrame(geometry=gpd.GeoSeries(points),crs='EPSG:4326') 
    #new_data['lon'] = new_lon
    #new_data['lat'] = new_lat
    gdf[col_name] = new_val

    return gdf

In [29]:
fires = gpd.read_file('site_selection/fire_extract_2014_plus_correct.shp')
fire_df = join_shapefile(sites_df,fires,'fire_detected',fire=True)
print(fire_df[fire_df['fire_detected'] >= 1])

                           geometry        lat        lon  fire_detected
220245   POINT (-80.70997 45.84868)  45.848684 -80.709966              1
220246   POINT (-80.70376 45.84766)  45.847657 -80.703764              1
220247   POINT (-80.69756 45.84663)  45.846630 -80.697561              1
221400   POINT (-80.70853 45.85312)  45.853118 -80.708528              1
221401   POINT (-80.70233 45.85209)  45.852092 -80.702325              1
...                             ...        ...        ...            ...
1221655  POINT (-79.18732 49.63519)  49.635187 -79.187322              1
1221656  POINT (-79.18062 49.63407)  49.634072 -79.180622              1
1222811  POINT (-79.17891 49.63843)  49.638435 -79.178907              1
1222812  POINT (-79.17221 49.63732)  49.637320 -79.172206              1
1223967  POINT (-79.17049 49.64168)  49.641682 -79.170490              1

[2018 rows x 4 columns]


In [30]:
cumulative_activity7 = gpd.read_file('site_selection/cumulative_activity_7yrs.shp')
ca7_df = join_shapefile(fire_df,cumulative_activity7,'ca7_detected',fire=False)

cumulative_activity6 = gpd.read_file('site_selection/cumulative_activity_6yrs.shp')
ca6_df = join_shapefile(ca7_df,cumulative_activity6,'ca6_detected',fire=False)

cumulative_activity5 = gpd.read_file('site_selection/cumulative_activity_5yrs.shp')
ca5_df = join_shapefile(ca6_df,cumulative_activity5,'ca5_detected',fire=False)

cumulative_activity4 = gpd.read_file('site_selection/cumulative_activity_4yrs.shp')
ca4_df = join_shapefile(ca5_df,cumulative_activity4,'ca4_detected',fire=False)

cumulative_activity3 = gpd.read_file('site_selection/cumulative_activity_3yrs.shp')
ca3_df = join_shapefile(ca4_df,cumulative_activity3,'ca3_detected',fire=False)

cumulative_activity2 = gpd.read_file('site_selection/cumulative_activity_2yrs.shp')
ca2_df = join_shapefile(ca3_df,cumulative_activity2,'ca2_detected',fire=False)

cumulative_activity1 = gpd.read_file('site_selection/cumulative_activity_1yrs.shp')
ca1_df = join_shapefile(ca2_df,cumulative_activity1,'ca1_detected',fire=False)

In [31]:
def get_species(gdf,interpolated_surface,transform,\
                    size,srcds,col_name):
    '''This is a function to get the value inside the fire.
    We will use to calculate the mean, median, max value for a fire.
    
    Parameters
    ----------
        gdf : GeoPandas DataFrame
            gdf containing potential site points
        interpolated_surface : ndarray
            an array of values in the study area
        transform : list 
            list describing GeoTransform of raster 
        size : list 
            pixel dimensions
        srcds : GDAL object
            read in raster
            
    Returns
    ----------
        gdf 
            GeoPandas DataFrame with species values as column
    '''


    #Get information about the raster - what is pixel size? What is the origin?
    xOrigin = transform[0]
    yOrigin = transform[3]
    xMax = xOrigin + transform[1] * size[0]
    yMin = yOrigin + transform[5] * size[1]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    value = []
    lon = []
    lat = [] 
    
    # Iterate through the points within the fire 
    for idx,p in gdf.iterrows():
        #mx , my is the lon lat 
        mx,my=np.array(p['geometry'].coords.xy[0])[0], np.array(p['geometry'].coords.xy[1])[0]
        col = int((mx - xOrigin) / pixelWidth)
        row = int((yOrigin - my ) / pixelHeight) 
        # Calculate offset on the array 
        if row < interpolated_surface.shape[0] and col < interpolated_surface.shape[1]: 
            # Index the array where the point is and send to list to store it 
            value.append(interpolated_surface[row][col])
        else: 
            value.append(np.nan)

        lon.append(mx)
        lat.append(my)

    #Add values in list to the dataframe 
    gdf[col_name] = value

    return gdf

In [32]:
# Open selected raster using gdal 
src_ds = gdal.Open('site_selection/beaudoin_age.tif')
# Get band 1 
rb1=src_ds.GetRasterBand(1)
transform=src_ds.GetGeoTransform()
cols = src_ds.RasterXSize
rows = src_ds.RasterYSize

# Convert raster to array 
data = rb1.ReadAsArray(0, 0, cols, rows)
df_age = get_species(ca1_df,data,transform,(cols,rows,),src_ds,'age')
print(df_age)
df_age = df_age.dropna(how='any')

                           geometry        lat  ...  ca1_detected         age
0        POINT (-85.90129 45.67832)  45.678324  ...             0   59.113335
1        POINT (-85.89502 45.67764)  45.677642  ...             0  103.583336
2        POINT (-85.88875 45.67696)  45.676960  ...             0   62.509998
3        POINT (-85.88247 45.67628)  45.676278  ...             0   74.239998
4        POINT (-85.87620 45.67559)  45.675594  ...             0   85.160004
...                             ...        ...  ...           ...         ...
1267030  POINT (-76.92042 49.41249)  49.412493  ...             0         NaN
1267031  POINT (-76.91381 49.41123)  49.411231  ...             0         NaN
1267032  POINT (-76.90721 49.40997)  49.409969  ...             0         NaN
1267033  POINT (-76.90060 49.40871)  49.408706  ...             0         NaN
1267034  POINT (-76.89400 49.40744)  49.407443  ...             0         NaN

[1267035 rows x 12 columns]


In [33]:
# Open selected raster using gdal 
src_ds = gdal.Open('site_selection/beaudoin_bf.tif')
# Get band 1 
rb1=src_ds.GetRasterBand(1)
transform=src_ds.GetGeoTransform()
cols = src_ds.RasterXSize
rows = src_ds.RasterYSize

# Convert raster to array 
data = rb1.ReadAsArray(0, 0, cols, rows)
df_bf = get_species(df_age,data,transform,(cols,rows,),src_ds,'Bf')
print(df_bf)
#df_age = df_age.dropna(how='any')

                           geometry        lat  ...         age         Bf
0        POINT (-85.90129 45.67832)  45.678324  ...   59.113335   1.303333
1        POINT (-85.89502 45.67764)  45.677642  ...  103.583336   3.403333
2        POINT (-85.88875 45.67696)  45.676960  ...   62.509998   4.666667
3        POINT (-85.88247 45.67628)  45.676278  ...   74.239998   5.573333
4        POINT (-85.87620 45.67559)  45.675594  ...   85.160004  16.143333
...                             ...        ...  ...         ...        ...
1266794  POINT (-78.48960 49.69822)  49.698216  ...    0.000000   0.000000
1266795  POINT (-78.48291 49.69706)  49.697057  ...    0.000000   0.000000
1266796  POINT (-78.47622 49.69590)  49.695897  ...    0.000000   0.000000
1266797  POINT (-78.46953 49.69474)  49.694736  ...    0.000000   0.000000
1266798  POINT (-78.46284 49.69358)  49.693575  ...   99.103333   0.886667

[1061034 rows x 13 columns]


In [34]:
# Open selected raster using gdal 
src_ds = gdal.Open('site_selection/beaudoin_sw.tif')
# Get band 1 
rb1=src_ds.GetRasterBand(1)
transform=src_ds.GetGeoTransform()
cols = src_ds.RasterXSize
rows = src_ds.RasterYSize

# Convert raster to array 
data = rb1.ReadAsArray(0, 0, cols, rows)
df_sw = get_species(df_bf,data,transform,(cols,rows,),src_ds,'Sw')
print(df_sw)

                           geometry        lat  ...         Bf        Sw
0        POINT (-85.90129 45.67832)  45.678324  ...   1.303333  1.870000
1        POINT (-85.89502 45.67764)  45.677642  ...   3.403333  0.380000
2        POINT (-85.88875 45.67696)  45.676960  ...   4.666667  3.033333
3        POINT (-85.88247 45.67628)  45.676278  ...   5.573333  4.336667
4        POINT (-85.87620 45.67559)  45.675594  ...  16.143333  3.213333
...                             ...        ...  ...        ...       ...
1266794  POINT (-78.48960 49.69822)  49.698216  ...   0.000000  0.000000
1266795  POINT (-78.48291 49.69706)  49.697057  ...   0.000000  0.000000
1266796  POINT (-78.47622 49.69590)  49.695897  ...   0.000000  0.000000
1266797  POINT (-78.46953 49.69474)  49.694736  ...   0.000000  0.000000
1266798  POINT (-78.46284 49.69358)  49.693575  ...   0.886667  0.780000

[1061034 rows x 14 columns]


In [35]:
# Open selected raster using gdal 
src_ds = gdal.Open('site_selection/beaudoin_sb.tif')
# Get band 1 
rb1=src_ds.GetRasterBand(1)
transform=src_ds.GetGeoTransform()
cols = src_ds.RasterXSize
rows = src_ds.RasterYSize

# Convert raster to array 
data = rb1.ReadAsArray(0, 0, cols, rows)
df_sb = get_species(df_sw,data,transform,(cols,rows,),src_ds,'Sb')
print(df_sb)

                           geometry        lat  ...        Sw       Sb
0        POINT (-85.90129 45.67832)  45.678324  ...  1.870000     0.00
1        POINT (-85.89502 45.67764)  45.677642  ...  0.380000     0.00
2        POINT (-85.88875 45.67696)  45.676960  ...  3.033333     1.09
3        POINT (-85.88247 45.67628)  45.676278  ...  4.336667     0.00
4        POINT (-85.87620 45.67559)  45.675594  ...  3.213333     1.30
...                             ...        ...  ...       ...      ...
1266794  POINT (-78.48960 49.69822)  49.698216  ...  0.000000 -9999.00
1266795  POINT (-78.48291 49.69706)  49.697057  ...  0.000000 -9999.00
1266796  POINT (-78.47622 49.69590)  49.695897  ...  0.000000 -9999.00
1266797  POINT (-78.46953 49.69474)  49.694736  ...  0.000000 -9999.00
1266798  POINT (-78.46284 49.69358)  49.693575  ...  0.780000 -9999.00

[1061034 rows x 15 columns]


In [36]:
#Get 7 dataframes for the 7 sites
df_sb = df_sb.dropna(how='any')
site1 = df_sb[df_sb['ca1_detected'] == 1]
print(site1)


                           geometry        lat  ...        Sw         Sb
102247   POINT (-82.01557 45.58011)  45.580106  ...  2.350000   1.623333
105712   POINT (-82.01164 45.59348)  45.593477  ...  1.520000   1.870000
109176   POINT (-82.01390 45.60779)  45.607789  ...  2.720000  21.879999
111485   POINT (-82.01748 45.61764)  45.617643  ...  1.406667   0.000000
113760   POINT (-82.23204 45.65923)  45.659228  ...  1.520000   0.000000
...                             ...        ...  ...       ...        ...
1147418  POINT (-81.42405 49.68505)  49.685050  ...  0.456667  48.910000
1147419  POINT (-81.41729 49.68408)  49.684081  ...  0.000000  80.239998
1148573  POINT (-81.42256 49.68945)  49.689447  ...  0.456667  75.793335
1148574  POINT (-81.41580 49.68848)  49.688478  ...  2.550000  74.779999
1152119  POINT (-80.87119 49.62270)  49.622698  ...  0.000000  93.176666

[18005 rows x 15 columns]


In [52]:
harvest = gpd.read_file('site_selection/harvest_buffer.shp')

harvest_df = join_shapefile(df_sb,harvest,'harvest_detected',harvest=True)
print(harvest_df[harvest_df['harvest_detected'] >= 1])

                           geometry        lat  ...        Sb  harvest_detected
279810   POINT (-83.75043 46.53814)  46.538141  ...  3.333333               1.0
279811   POINT (-83.74408 46.53732)  46.537318  ...  0.000000               1.0
280965   POINT (-83.74926 46.54261)  46.542608  ...  0.000000               1.0
280966   POINT (-83.74290 46.54178)  46.541785  ...  0.000000               1.0
282151   POINT (-83.55121 46.52135)  46.521352  ...  0.000000               1.0
...                             ...        ...  ...       ...               ...
1201535  POINT (-82.49888 50.04871)  50.048713  ...  0.000000               1.0
1202689  POINT (-82.50433 50.05402)  50.054018  ...  0.000000               1.0
1202690  POINT (-82.49749 50.05312)  50.053121  ...  0.000000               1.0
1203844  POINT (-82.50294 50.05843)  50.058426  ...  0.000000               1.0
1203845  POINT (-82.49609 50.05753)  50.057528  ...  0.000000               1.0

[22384 rows x 16 columns]


In [63]:
road = gpd.read_file('site_selection/road_buffer_1km.shp')

road_df = join_shapefile(harvest_df,road,'road_detected',road=True)
print(road_df[road_df['road_detected'] == 1])

                           geometry        lat  ...  harvest_detected  road_detected
7106     POINT (-84.79375 45.57881)  45.578811  ...               0.0              1
7107     POINT (-84.78751 45.57805)  45.578055  ...               0.0              1
7108     POINT (-84.78126 45.57730)  45.577298  ...               0.0              1
7109     POINT (-84.77502 45.57654)  45.576541  ...               0.0              1
8261     POINT (-84.79270 45.58330)  45.583303  ...               0.0              1
...                             ...        ...  ...               ...            ...
1204177  POINT (-80.24066 49.73488)  49.734882  ...               0.0              1
1204178  POINT (-80.23392 49.73384)  49.733836  ...               0.0              1
1205330  POINT (-80.25253 49.74135)  49.741351  ...               0.0              1
1205331  POINT (-80.24579 49.74031)  49.740306  ...               0.0              1
1205332  POINT (-80.23905 49.73926)  49.739261  ...              

In [13]:
def get_dist_driving(G, gdf1,gdf2): 
    site_options = {} 
    for x,y in zip(gdf1['lon'],gdf1['lat']):
        df = pd.DataFrame() 
        dist_list = [] 
        lon_list = [] 
        lat_list = [] 
        for x2,y2 in zip(gdf2['lon'],gdf2['lat']): 
            if (x,y,) != (x2,y2): 

                # get the nearest network node to each point
                orig_node = ox.get_nearest_node(G, (y,x))
                dest_node = ox.get_nearest_node(G, (y2,x2))

                # how long is our route in meters?
                d = nx.shortest_path_length(G, orig_node, dest_node, weight='length')
                dist_list.append(d)
                lon_list.append(x2)
                lat_list.append(y2)

        df['dist'] = dist_list
        df['lon'] = lon_list
        df['lat'] = lat_list
        site_options[(x,y,)] = df

import math 
def get_dist_haversine(gdf1,gdf2): 
    site_options = {} 
    for x,y in zip(gdf1['lon'],gdf1['lat']):
        df = pd.DataFrame() 
        dist_list = [] 
        lon_list = [] 
        lat_list = [] 
        for x2,y2 in zip(gdf2['lon'],gdf2['lat']): 
            if (x,y,) != (x2,y2): 

                lon1, lat1, lon2, lat2 = map(math.radians, [x, y, x2, y2])

                # haversine formula 
                dlon = lon2 - lon1 
                dlat = lat2 - lat1 
                a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
                c = 2 * math.asin(math.sqrt(a)) 
                r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
                dist = c * r
                dist_list.append(dist)
                lon_list.append(x2)
                lat_list.append(y2)

        df['dist'] = dist_list
        df['lon'] = lon_list
        df['lat'] = lat_list
        site_options[(x,y,)] = df

    return site_options

In [65]:
df_all = road_df.dropna(how='any')
df_all = df_all[df_all['fire_detected'] == 0]
df_all = df_all[df_all['harvest_detected'] == 0]
#df_all = df_all[df_all['road_detected'] == 1]
df_all['combo_species'] = df_all['Bf'] + df_all['Sw'] +df_all['Sb']
df_all = df_all[df_all['combo_species'] >= 30]
df_all = df_all[df_all['age'] >= 30]

print(df_all)

site1 = df_all[df_all['ca1_detected'] == 1]
site1 = site1[site1['ca2_detected'] == 0]
site1 = site1[site1['ca3_detected'] == 0]
site1 = site1[site1['ca4_detected'] == 0]
site1 = site1[site1['ca5_detected'] == 0]
site1 = site1[site1['ca6_detected'] == 0]
site1 = site1[site1['ca7_detected'] == 0]

print(site1)

site2 =  df_all[df_all['ca2_detected'] == 1]
site2 = site2[site2['ca3_detected'] == 0]
site2 = site2[site2['ca4_detected'] == 0]
site2 = site2[site2['ca5_detected'] == 0]
site2 = site2[site2['ca6_detected'] == 0]
site2 = site2[site2['ca7_detected'] == 0]
print(site2)

site3 =  df_all[df_all['ca3_detected'] == 1]
site3 = site3[site3['ca4_detected'] == 0]
site3 = site3[site3['ca5_detected'] == 0]
site3 = site3[site3['ca6_detected'] == 0]
site3 = site3[site3['ca7_detected'] == 0]
print(site3)

site4 =  df_all[df_all['ca4_detected'] == 1]
site4 = site4[site4['ca5_detected'] == 0]
site4 = site4[site4['ca6_detected'] == 0]
site4 = site4[site4['ca7_detected'] == 0]

print(site4)

site5 =  df_all[df_all['ca5_detected'] == 1]
site5 = site5[site5['ca6_detected'] == 0]
site5 = site5[site5['ca7_detected'] == 0]
print(site5)

site6 =  df_all[df_all['ca6_detected'] == 1]
site6 = site6[site6['ca7_detected'] == 0]
print(site6)

site7 =  df_all[df_all['ca7_detected'] == 1]
print(site7)


                           geometry        lat  ...  road_detected  combo_species
6        POINT (-85.86366 45.67423)  45.674227  ...              0      46.486668
7        POINT (-85.85738 45.67354)  45.673543  ...              0      49.396664
21       POINT (-85.76959 45.66392)  45.663919  ...              0      47.470001
31       POINT (-85.70691 45.65699)  45.656994  ...              0      31.066666
32       POINT (-85.70064 45.65630)  45.656299  ...              0      40.503334
...                             ...        ...  ...            ...            ...
1266671  POINT (-79.31538 49.83752)  49.837520  ...              0      47.779999
1266691  POINT (-79.18075 49.81532)  49.815321  ...              0      37.880001
1266719  POINT (-78.99249 49.78395)  49.783947  ...              0      35.360001
1266724  POINT (-78.95891 49.77831)  49.778308  ...              0      40.040001
1266748  POINT (-78.79781 49.75109)  49.751090  ...              0      30.333334

[430737 rows x 

In [54]:
dd1 = get_dist_haversine(site7,site1) #There are the least site 7's so we optimize distance to them
dd2 = get_dist_haversine(site7,site2)
dd3 = get_dist_haversine(site7,site3)
dd4 = get_dist_haversine(site7,site4)
dd5 = get_dist_haversine(site7,site5)
dd6 = get_dist_haversine(site7,site6)

In [55]:

def optimize_for_site(site,dict1,dict2,dict3,dict4,dict5,dict6):

    information = pd.DataFrame() 

    pot_site1 = dict1[site].reset_index(drop=True)
    pot_site2 = dict2[site].reset_index(drop=True)
    pot_site3 = dict3[site].reset_index(drop=True)
    pot_site4 = dict4[site].reset_index(drop=True)
    pot_site5 = dict5[site].reset_index(drop=True)
    pot_site6 = dict6[site].reset_index(drop=True)

    solver = pywraplp.Solver.CreateSolver('SCIP')

    var_list = []
    dist_list = [] 
    lon_list = [] 
    lat_list = [] 
    var_list_s1 = [] 
    var_list_s2 = []
    var_list_s3 = []
    var_list_s4 = []
    var_list_s5 = []
    var_list_s6 = []

    count = 0 
    for list_ in [pot_site1,pot_site2,pot_site3,\
                  pot_site4,pot_site5,pot_site6]:
        count+=1 
        count2 = 0 #count 2 is the site index 
        for inst in list_.iterrows(): 
            inst = list(inst)[1]
            var_list.append('x'+str(count)+'_'+str(count2))
            dist_list.append(inst['dist'])
            lon_list.append(inst['lon'])
            lat_list.append(inst['lat'])
            if count == 1: 
                var_list_s1.append('x'+str(count)+'_'+str(count2))
            if count == 2: 
                var_list_s2.append('x'+str(count)+'_'+str(count2))
            if count == 3: 
                var_list_s3.append('x'+str(count)+'_'+str(count2))
            if count == 4: 
                var_list_s4.append('x'+str(count)+'_'+str(count2))           
            if count == 5: 
                var_list_s5.append('x'+str(count)+'_'+str(count2))
            if count == 6: 
                var_list_s6.append('x'+str(count)+'_'+str(count2))
            count2+=1 

    var_collect = {}

    for i in range(0, len(var_list)): 
        # solver.IntVar(0.0, 1, var_list[i]) --> tell computer that the variable can
        # only be 1 or 0. 
        var_collect[var_list[i]] = solver.IntVar(0.0, 1, var_list[i]) 
                
    information['variable'] = var_list
    information['distance'] = dist_list 
    information['lat'] = lat_list
    information['lon'] = lon_list

    locals().update(var_collect)
    globals().update(var_collect)
    vars = [eval(i) for i in var_list]
    vars1 = [eval(i) for i in var_list_s1]
    vars2 = [eval(i) for i in var_list_s2]
    vars3 = [eval(i) for i in var_list_s3]
    vars4 = [eval(i) for i in var_list_s4]
    vars5 = [eval(i) for i in var_list_s5]
    vars6 = [eval(i) for i in var_list_s6]

    solver.Add(sum(vars1) == 1)
    solver.Add(sum(vars2) == 1)
    solver.Add(sum(vars3) == 1)
    solver.Add(sum(vars4) == 1)
    solver.Add(sum(vars5) == 1)
    solver.Add(sum(vars6) == 1)

    #print('Number of constraints =', solver.NumConstraints()) 
    #print('Number of variables =', solver.NumVariables())

    #Tell computer that each component of the objective function is dist * variable
    obj_collect = [i*j for i, j in zip(vars,dist_list)]
    #Tell the computer to maximize the sum of these volume*variable pairs 
    solver.Minimize(sum(obj_collect))

    #Solve 
    status = solver.Solve()

    #Get information from solver 
    decision = [] 
    if status == pywraplp.Solver.OPTIMAL:
        
        #print('Objective value =', solver.Objective().Value())
        for var in vars: 
          #Append to decision list the solution for the specific variable. 
          decision.append(abs(var.solution_value()))
    else:
        print('The problem does not have an optimal solution.')

    if len(decision) > 0: 
      #Append a row to the original table IF there is an optimal solution
      information['visit'] = decision 

      #Here's our table. 'cut' tells us whether to harvest or not. 
      #print(information[information['visit'] == 1])
    information = information[information['visit'] == 1]
    return float(solver.Objective().Value()),information


In [90]:
op_list = [] 
mdist_list = [] 
site7_loc = [] 
for site in site7.iterrows(): 
    site = list(site)[1]
    loc = (site['lon'],site['lat'],)
    obj_func, info = optimize_for_site(loc,dd1,dd2,dd3,dd4,dd5,dd6)
    op_list.append(info)
    mdist_list.append(obj_func)
    site7_loc.append(loc)

#get min

index_min = mdist_list.index(min(mdist_list))

print('Selected Sites')
print(op_list[index_min])
print('Selected Site 7 Location')
print(site7_loc[index_min])
print('Minimum Haversine Dist Possible Bt Sites (km)')
print(mdist_list[index_min])

df_sites = op_list[index_min]

Selected Sites
      variable  distance        lat        lon  visit
3175   x1_3175  0.705016  48.651504 -82.505391    1.0
9711    x2_618  0.497070  48.654127 -82.490783    1.0
12185   x3_116  1.410031  48.647980 -82.513369    1.0
12535    x4_20  3.512921  48.624059 -82.506866    1.0
12802    x5_19  0.501829  48.650604 -82.498763    1.0
12842     x6_8  2.914012  48.674441 -82.470757    1.0
Selected Site 7 Location
(-82.49741145629419, 48.65502796480138)
Minimum Haversine Dist Possible Bt Sites (km)
9.540880545417


In [91]:
df_sites = df_sites.append(pd.DataFrame([['x0', 0,site7_loc[index_min][1],\
                               site7_loc[index_min][0],1.0]],columns=df_sites.columns))


df_sites['years_act'] = [1,2,3,4,5,6,7]

print(df_sites)
gdf_sites = gpd.GeoDataFrame(df_sites,geometry=\
                             gpd.points_from_xy(df_sites['lon'],df_sites['lat']))
gdf_sites.to_file(driver = 'ESRI Shapefile',filename='op_sites2')
gdf_sites.to_file(driver='GeoJSON',filename='geojson_op_sites2')

      variable  distance        lat        lon  visit  years_act
3175   x1_3175  0.705016  48.651504 -82.505391    1.0          1
9711    x2_618  0.497070  48.654127 -82.490783    1.0          2
12185   x3_116  1.410031  48.647980 -82.513369    1.0          3
12535    x4_20  3.512921  48.624059 -82.506866    1.0          4
12802    x5_19  0.501829  48.650604 -82.498763    1.0          5
12842     x6_8  2.914012  48.674441 -82.470757    1.0          6
0           x0  0.000000  48.655028 -82.497411    1.0          7
